In [7]:
from data_loading.loo_data_loader import DesignatedDataLoader
data_loader = DesignatedDataLoader.load_from(
    path='../data_loading/mindreader',
    movies_only=True,
    min_num_entity_ratings=1,
    filter_unknowns=True
)

data_loader.random_seed = 345

print(data_loader.info())

train, validation, test = data_loader.make(
    movie_to_entity_ratio=1,
    n_negative_samples=99
)

Init dataloader with 17652 ratings
 
            DataLoader Information
            -----------------------------------
            n_users:                      929
            n_movies:                     2993
            n_descriptive_entities:       0

            n_ratings:                    17652
            n_movie_ratings:              17652
            n_descriptive_entity_ratings: 0
        
Asserting positive samples not in training set for each user...
Asserting negative samples occurrence in training set, but not rated for each user...
Asserting positive samples do not occur in negative samples...
Returning a dataset over 927 users.


In [1]:
from collections import defaultdict

def get_top_pop(train_list):
    movie_count = defaultdict(int)

    for user, ratings in train_list:
        for rating in ratings:
            movie_count[rating.e_idx] += 1

    return movie_count

def rank(items, top_pop):
    items = set(items)
    item_scores = {item: score for item, score in top_pop.items() if item in items}
    
    return sorted(item_scores.items(), key=lambda item: item[1], reverse=True)

top = get_top_pop(train)
k = 10

def evaluate(eval_list):
    hits = 0
    count = 0
    
    for user, test_neg in eval_list:
        to_hit = test_neg[0]
        
        ranks = [item[0] for item in rank([to_hit] + test_neg[1], top)[:k]]
        if to_hit in ranks:
            hits += 1
        count += 1
        
    return hits, count
        

h, c = evaluate(test)

print(f'{h / c * 100}%')

NameError: name 'train' is not defined